In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import matplotlib as mpl
from scipy import signal
from scipy.interpolate import CubicSpline
import glob

In [ ]:
%matplotlib qt

In [ ]:
EXPERIMENT = 'J049'

In [ ]:
data = pd.DataFrame({'HA': [], 'LA': []})
for filename in glob.glob('T:/Team/Szewczyk/Juliane/J049/*.txt'):
    print(filename)
    data_partial = pd.read_csv(filename, names=['t', 'HA', 'LA'], sep='\t')
    data_partial = data_partial.drop('t', axis=1)
    data = pd.concat([data, data_partial]).reset_index(drop=True)
data['t'] = np.arange(data.HA.size)

T:/Team/Szewczyk/Juliane/J049\J04906_0-1800_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04906_1800-3600_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04907_0-1800_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04907_1800-3600_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04908_0-1800_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04908_1800-3600_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04909_0-1800_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04909_1800-3600_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04910_0-1800_ts.txt
T:/Team/Szewczyk/Juliane/J049\J04910_1800-3600_ts.txt


In [ ]:
data

,HA,LA,t
0,-0.524642,-0.862368,0
1,-0.276372,-0.897157,1
2,-0.085724,-0.939779,2
3,0.087276,-0.979031,3
4,0.241856,-1.016433,4
...,...,...,...
17985,-0.153203,-0.169093,17985
17986,-0.143994,-0.150863,17986
17987,-0.130932,-0.141042,17987
17988,-0.107966,-0.130426,17988


In [ ]:
sos = signal.butter(20, 0.1, fs=1 ,output='sos')
data['HA_filt'] = signal.sosfiltfilt(sos, data.HA)
data['LA_filt'] = signal.sosfiltfilt(sos, data.LA)

In [ ]:
cs1 = CubicSpline(data.t, data.HA_filt)
cs2 = CubicSpline(data.t, data.LA_filt)
new_time = np.arange(0, data.t.iloc[-1], 0.01)
fit = pd.DataFrame({
    't'     : new_time,
    'HA'   : cs1(new_time),
    'LA'   : cs2(new_time),
})

In [ ]:
peaksHA = signal.find_peaks(fit.HA[500:], height=0, distance=1000)[0]+500
peaksLA = signal.find_peaks(fit.LA[500:], height=0, distance=1000)[0]+500

In [ ]:
# peaksHA = np.delete(peaksHA, 41)

In [ ]:
plt.figure()
plt.plot(fit.t, fit.HA, label = 'electrode 1')
plt.plot(fit.t, fit.LA, label = 'electrode 2')
plt.title(f'{EXPERIMENT} waveforms')
plt.xlabel('t [s]')
plt.ylabel(r'$\xi$')
# plt.legend()
plt.scatter(fit.t[peaksLA], fit.LA[peaksLA], c='r', marker='x')
plt.scatter(fit.t[peaksHA], fit.HA[peaksHA], c='g', marker='x')

In [ ]:
fig, axs = plt.subplots(2, sharex=True)
axs[0].plot(fit.t, fit.HA, label = 'HA')
axs[0].plot(fit.t, fit.LA, label = 'LA')
fig.suptitle(f'{EXPERIMENT} waveforms')
axs[0].set_ylabel(r'$\xi$')
# plt.legend()
# axs[0].scatter(fit.t[peaksLA], fit.LA[peaksLA], c='r', marker='x')
axs[1].plot(peaksHA[1:]/100, 2*np.diff(peaksHA)/100)
axs[1].plot(peaksLA[1:]/100, np.diff(peaksLA)/100)
axs[1].plot(peaksHA[1:]/100, np.diff(peaksHA)/100)
axs[1].set_xlabel("t [s]")
axs[1].set_ylabel("Period [s]")
# axs[1].plot(peaksHA[1::2]/100, np.diff(peaksHA)[::2]/100)
# axs[1].plot(peaksHA[2::2]/100, np.diff(peaksHA)[1::2]/100)

Text(0, 0.5, 'Period [s]')

In [ ]:
first_HA_peak_to_left = np.searchsorted(peaksHA, peaksLA, side="left")
first_HA_peak_to_right = np.searchsorted(peaksHA, peaksLA, side="right")
peak_distances = (peaksHA[first_HA_peak_to_left] - peaksLA)/np.diff(peaksHA, append=0)[first_HA_peak_to_left]

In [ ]:
fig, axs = plt.subplots(2, sharex=True)
axs[0].plot(fit.t, fit.HA, label = 'HA')
axs[0].scatter(fit.t[peaksHA], fit.HA[peaksHA], c='r', marker='x')
axs[0].plot(fit.t, fit.LA, label = 'LA')
axs[0].scatter(fit.t[peaksLA], fit.LA[peaksLA], c='r', marker='x')
fig.suptitle(f'{EXPERIMENT} waveforms')
axs[0].set_ylabel(r'$\xi$')

# axs[1].plot(peaksLA[:220]/100, (peaksHA[:440:2]-peaksLA[:220])/np.diff(peaksHA[:440:2], prepend=0))
axs[1].scatter(peaksLA/100, peak_distances, marker = '+', color = 'green')
axs[1].set_ylabel(r"Phase difference (units of $\mathregular{T_{HA}}$)")

Text(0, 0.5, 'Phase difference (units of $\\mathregular{T_{HA}}$)')

In [ ]:
plt.figure()
plt.plot(peaksLA[:200]/100, (peaksHA[:400:2]-peaksLA[:200])/np.diff(peaksHA[:400:2], prepend=0))
for i in [-0.5, 0.5, 4.5, 14, 15]:
    plt.axhline(i, ls='--', c='gray')

In [ ]:
csp1 = CubicSpline(peaksHA/100, range(peaksHA.shape[0]))
fit['phase1'] = csp1(fit.t)
csp2 = CubicSpline(peaksLA/100, range(peaksLA.shape[0]))
fit['phase2'] = csp2(fit.t)

In [ ]:
plt.figure()
plt.title(f'Phase difference {EXPERIMENT}')
plt.xlabel('t [s]')
plt.ylabel(r'$\Delta \phi$')
plt.axhline(0, ls='--')
plt.plot(fit.t, fit.phase1 - fit.phase2)

In [ ]:
plt.figure()
plt.plot(fit.phase1.iloc[peaksLA]%1)

In [ ]:
plt.plot(fit.t, fit.HA)
plt.plot(fit.t, fit.phase1%1*5+70)

In [ ]:
plt.figure()
plt.title(f'LA synchronisation {EXPERIMENT}')

plt.xlabel('t [s]')
plt.ylabel('T [s]')


plt.plot(fit.t[peaksHA][1:], np.diff(fit.t[peaksHA]), label='electrode 1')
plt.plot(fit.t[peaksLA][1:], np.diff(fit.t[peaksLA]), label='electrode 2')
plt.legend()